In [1]:
import os
import glob
import pandas as pd
import csv

In [2]:
os.chdir("../data/")

#  Use glob to match the pattern ‘csv’
dir1 = 'Summary_Details/202*/202*.'
dir2 = 'Summary_Hashtag/202*/202*.'
dir3 = 'Summary_Sentiment/202*/202*.'
file_type = 'csv'
all_summ = [i for i in glob.glob(dir1 + file_type)]
all_hash = [i for i in glob.glob(dir2 + file_type)]
all_sent = [i for i in glob.glob(dir3 + file_type)]
all_hash[0]

## define the file scope
files = all_summ

## define start date
startTime = '2020-01-01 00:00:00'
endTime = '2020-01-31 00:00:00'

## define start and end time
startTime = pd.to_datetime(startTime)
endTime = pd.to_datetime(endTime)

In [23]:
####-----------------------FOR Summary Geo--------------#######
# pd.merge(product,customer,on='Product_ID')
## conbine file in certain interval

def get_ratio(df, year, month):
        d = {}
        po_cnt = df[df['Sentiment_Label'] == 'positive'].shape[0]
        ne_cnt = df[df['Sentiment_Label'] == 'negative'].shape[0]
        total_cnt = float(df.shape[0])
        d['ratio'] = (po_cnt-ne_cnt)/total_cnt
        d['total'] = total_cnt
        d['year'] = year
        d['month'] = month
        return pd.Series(d, index=['ratio','total','year','month'])
    
def write_res(files, start, end):
    startTime = pd.to_datetime(start)
    endTime = pd.to_datetime(end)
    
    res = []
    for x in files:
        temp = '_'.join(x.split('/')[2].split('_',4)[:-1])                    
        date = pd.to_datetime(temp,  format='%Y_%m_%d_%H')


        if date >= startTime and date <= endTime:
            summ = pd.read_csv(x,encoding= 'unicode_escape')
            f = "Summary_Sentiment/" + temp.split('_')[0] + "_" + temp.split('_')[1] + "/" + temp + "_Summary_Sentiment.csv"
            if os.path.exists(f):
                sent = pd.read_csv(f, encoding="unicode_escape")
    #         hashtag = pd.read_csv("Summary_Hashtag/2020_" + temp.split('_')[1] + "/" +
    #                            temp + "_Summary_Hashtag.csv", encoding="unicode_escape")
            geo = summ[summ['Country'].notnull()]

            if not geo.empty:
                temp = pd.merge(geo[['Tweet_ID','Language','Country']], sent[['Tweet_ID','Sentiment_Label']], how='inner', on='Tweet_ID')
                temp['year'] = date.year
                temp['month'] = date.month
                temp['day'] = date.day
                res.append(temp)


    res = pd.concat(res, ignore_index=True)
    
    #export combined file to csv
    res.to_csv('combined_res_from_{}_to_{}.csv'.format(startTime.date(),endTime.date()), 
                        index=False, encoding="utf-8",errors='ignore')

    

    s1 = res[['Country', 'Sentiment_Label', 'Tweet_ID', 'year', 'month']]
    s1 = s1.groupby(['Country']).apply(lambda x: get_ratio(x, startTime.year, startTime.month)).reset_index()
    s1['year'] = s1['year'].astype(int)
    s1['month'] = s1['month'].astype(int)
    
#     country = pd.read_csv("country_codes.csv")[['code','area','long','lat']]
#     comb = s1.merge(country, how='left', left_on='Country', right_on="code").drop(['code'], axis=1)
    s1.to_csv('combined_res_from_{}_to_{}.csv'.format(startTime.date(),endTime.date()), 
                        index=False, encoding="utf-8",errors='ignore')
    print("Have write ", start)

Time = [('2020-01-01 00:00:00','2020-01-31 00:00:00'), ('2020-02-01 00:00:00','2020-02-29 00:00:00'),
       ('2020-03-01 00:00:00','2020-03-31 00:00:00'), ('2020-04-01 00:00:00','2020-04-30 00:00:00'),
       ('2020-05-01 00:00:00','2020-05-29 00:00:00'), ('2020-06-01 00:00:00','2020-06-30 00:00:00'),
       ('2020-07-01 00:00:00','2020-07-31 00:00:00'), ('2020-08-01 00:00:00','2020-08-31 00:00:00'),
       ('2020-09-01 00:00:00','2020-09-30 00:00:00'), ('2020-10-01 00:00:00','2020-10-31 00:00:00'),
       ('2020-11-01 00:00:00','2020-11-30 00:00:00'), ('2020-12-01 00:00:00','2020-12-31 00:00:00'),
       ('2021-01-01 00:00:00','2021-01-31 00:00:00'), ('2021-02-01 00:00:00','2021-02-28 00:00:00'),
       ('2021-03-01 00:00:00','2021-03-31 00:00:00'), ('2021-04-01 00:00:00','2021-04-30 00:00:00')]

for start,end in Time:
    write_res(files, start, end)

/rhome/xfeng044/bigdata/.conda/envs/stat209/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Have write  2020-01-01 00:00:00
Have write  2020-02-01 00:00:00
Have write  2020-03-01 00:00:00
Have write  2020-04-01 00:00:00
Have write  2020-05-01 00:00:00
Have write  2020-06-01 00:00:00
Have write  2020-07-01 00:00:00
Have write  2020-08-01 00:00:00
Have write  2020-09-01 00:00:00
Have write  2020-10-01 00:00:00
Have write  2020-11-01 00:00:00
Have write  2020-12-01 00:00:00
Have write  2021-01-01 00:00:00
Have write  2021-03-01 00:00:00
Have write  2021-04-01 00:00:00


In [26]:
# merge the res_files
all_files = glob.glob(os.path.join("./", "combined_res_from_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "res_merged.csv")